# Standard Imports

In [274]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# Constants

In [275]:
seed = 42

# Import data

In [276]:
# Chemical and Instrumental Assessment of Green Tea Sensory Preference - Y.R. Liang, Q. Ye, J. Jin, H. Liang, J.L. Lu, Y.Y. Du & J.J. Dong
# https://doi.org/10.1080/10942910701299430
# Notes: downloaded CSVs

df_10942910701299430_chemical_composition = pd.read_csv('../data/10942910701299430/chemical_composition.csv', index_col=0, header=1)
df_10942910701299430_sensory_evaluation = pd.read_csv('../data/10942910701299430/sensory_evaluation.csv')

# Phytochemical Composition and Antioxidant Capacity of 30 Chinese Teas - by Guo-Yi Tang, Cai-Ning Zhao, Xiao-Yu Xu, Ren-You Gan, Shi-Yu Cao, Qing Liu, AoShang, Qian-Qian Mao & Hua-Bin Li
# https://doi.org/10.3390/antiox8060180
# Notes: Manually scraped data

df_antiox8060180_chemical_composition = pd.read_csv('../data/antiox8060180/chemical_composition.csv')

# Catechin and caffeine content of tea (Camellia sinensis L.) leaf significantly differ with seasonal variation... - Himangshu Deka, Tupu Barman, Jintu Dutta, Arundhuti Devi, Pradip Tamuly, Ranjit Kumar Paul & Tanmoy Karak 
# https://doi.org/10.1016/j.jfca.2020.103684
# https://krishi.icar.gov.in/jspui/bitstream/123456789/68751/2/S0889157520313892-main.pdf - Free access

# df_j.jfca.2020.103684_chemical_composition = pd.read_csv('dava/j.jfca.2020.103684/chemical_composition.csv')

# Comparative analysis of tea catechins and theaflavins by high-performance liquid chromatography and capillary electrophoresis - Bee-Lan Lee & Choon-Nam Ong
# https://doi.org/10.1016/S0021-9673(00)00215-6
# Notes: Manually scraped data

df_S0021967300002156 = pd.read_csv('../data/S0021967300002156/chemical_composition.csv')

# Survey of Catechins, Gallic Acid, and Methylxanthines in Green, Oolong, Pu-erh, and Black Teas - Jen-Kun Lin, Chih-Li Lin, Yu-Chih Liang, Shoei-Yn Lin-Shiau & I-Ming Juan
# https://doi.org/10.1021/jf980223x
# Notes: Manually scraped data

df_S0021967300002156 = pd.read_csv('../data/jf980223x/chemical_composition.csv')


# Combine data

To combine the data I am going to create a data pipeline that does the following:

1. Add all the pandas dataframes to a list.
2. Read that list into a function.
3. The function reads an individual dataframe from that list then resets the index, rename catechins to be like the catechin dictionary below, drop unused columns, add columns that are part of the standardized column format, and then sorts the columns to be part of the standardized column format.
4. The function will then add the dataframe to a list denoting it is processed.
5. Once the function has processed all the dataframes in the lists it will loop through the data frames in the finished list and concatenate them.
6. The function will return a combined dataframe.

## Data Frame Format

The data frames will be combined into the following format:

- `'Catechin'`
- `'Epicatechin'`
- `'Gallocatechin'`
- `'Epigallocatechin'`
- `'Catechin Gallate'`
- `'Epicatechin Gallate'`
- `'Gallocatechin Gallate'`
- `'Epigallocatechin Gallate'`
- `'Gallic Acid'`
- `'Chlorogenic Acid'`
- `'Caffeine'`
- `'Taste'`
- `'Appearance'`
- `'Aroma'`
- `'Liqour color'`


In [277]:
catechin_dictionary = {
    'C': 'Catechin',
    'EC': 'Epicatechin',
    'GC': 'Gallocatechin',
    'EGC': 'Epigallocatechin',
    'CG': 'Catechin Gallate',
    'ECG': 'Epicatechin Gallate',
    'GCG': 'Gallocatechin Gallate',
    'EGCG': 'Epigallocatechin Gallate'
}

standardized_columns = [
    'Catechin',
    'Epicatechin',
    'Gallocatechin',
    'Epigallocatechin',
    'Catechin Gallate',
    'Epicatechin Gallate',
    'Gallocatechin Gallate',
    'Epigallocatechin Gallate',
    'Gallic Acid',
    'Chlorogenic Acid',
    'Caffeine',
    'Taste',
    'Appearance',
    'Aroma',
    'Liqour color'
]

In [278]:
df_10942910701299430_chemical_composition = df_10942910701299430_chemical_composition.reset_index(drop=1)
# df_10942910701299430_chemical_composition = df_10942910701299430_chemical_composition.drop(columns=['Nitrogen', 'Amino acids', 'Ascorbic acid', 'Polyphenols', 'Total catechins'])
df_10942910701299430_chemical_composition = df_10942910701299430_chemical_composition.rename(columns=catechin_dictionary)
df_10942910701299430_chemical_composition = df_10942910701299430_chemical_composition.drop(columns=[column for column in df_10942910701299430_chemical_composition if column not in standardized_columns])
df_10942910701299430_chemical_composition = df_10942910701299430_chemical_composition.reindex(columns=standardized_columns)
df_10942910701299430_chemical_composition

,Catechin,Epicatechin,Gallocatechin,Epigallocatechin,Catechin Gallate,Epicatechin Gallate,Gallocatechin Gallate,Epigallocatechin Gallate,Gallic Acid,Chlorogenic Acid,Caffeine,Taste,Appearance,Aroma,Liqour color
0,8.93 ± 0.09,7.04 ± 07,8.36 ± 0.08,18.00 ± 0.16,3.72 ± 0.04,4.66 ± 0.07,21.50 ± 0.38,21.11 ± 0.28,NaN,NaN,31.47 ± 0.22,NaN,NaN,NaN,NaN
1,9.87 ± 008,.4.99 ± 0.04,22.55 ± 0.21,10.92 ± 0.08,5.33 ± 0.04,5.29 ± 0.09,26.62 ± 0.34,23.42 ± 0.18,NaN,NaN,39.24 ± 0.18,NaN,NaN,NaN,NaN
2,6.37 ± 0.07,4.92 ± 0.06,21.87 ± 0.17,10.98 ± 0.08,5.29 ± 0.07,5.43 ± 0.07,24.84 ± 0.42,23.63 ± 0.21,NaN,NaN,41.08 ± 0.32,NaN,NaN,NaN,NaN
3,6.78 ± 0.03,5.91 ± 0.07,23.08 ± 0.18,11.47 ± 0.12,5.76 ± 0.06,6.43 ± 0.06,25.34 ± 0.27,22.14 ± 0.20,NaN,NaN,50.97 ± 0.29,NaN,NaN,NaN,NaN
4,5.82 ± 0.07,5.60 ± 0.07,24.12 ± 0.22,13.35 ± 0.11,5.28 ± 0.06,5.94 ± 0.04,27.92 ± 0.32,25.09 ± 0.22,NaN,NaN,53.70 ± 0.45,NaN,NaN,NaN,NaN
5,6.64 ± 0.08,8.98 ± 0.13,22.49 ± 0.23,12.18 ± 0.12,5.75 ± 0.08,7.36 ± 0.08,28.51 ± 0.19,31.18 ± 0.32,NaN,NaN,52.26 ± 0.36,NaN,NaN,NaN,NaN
6,5.77 ± 0.04,6.13 ± 0.07,18.14 ± 0.08,10.76 ± 0.09,4.85 ± 0.03,6.28 ± 0.09,23.35 ± 0.22,23.00 ± 0.18,NaN,NaN,58.83 ± 0.65,NaN,NaN,NaN,NaN
7,9.50 ± 0.12,9.15 ± 0.10,28.84 ± 0.26,15.16 ± 0.13,10.79 ± 0.04,14.57 ± 0.11,29.65 ± 0.45,34.13 ± 0.29,NaN,NaN,50.82 ± 0.29,NaN,NaN,NaN,NaN
8,7.77 ± 0.08,7.86 ± 0.16,26.27 ± 0.24,14.97 ± 0.16,6.22 ± 0.07,8.20 ± 0.10,27.05 ± 0.33,29.38 ± 0.26,NaN,NaN,35.65 ± 0.44,NaN,NaN,NaN,NaN
9,4.61 ± 0.05,5.66 ± 0.05,17.99 ± 0.15,11.99 ± 0.12,5.17 ± 0.09,7.38 ± 0.07,27.49 ± 0.31,29.87 ± 0.23,NaN,NaN,46.45 ± 0.35,NaN,NaN,NaN,NaN


# Visualize data

# Data Wrangling

# Training Device

In [279]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


# Generative Adversarial Network

In [280]:
class Generator(nn.Module):
    def __init__(self, ngpu):
            super(Generator, self).__init__()
            self.ngpu = ngpu

class Discriminator(nn.Module):    
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu

# Split data

In [281]:
from torch.utils.data import DataLoader, random_split

train_size = int(0.8 * len(df))
test_size = len(df) - train_size
train_df, test_df = random_split(df, [train_size, test_size])

train_loader = DataLoader(train_df, batch_size=64, shuffle=True)
test_loader = DataLoader(test_df, batch_size=64, shuffle=True)

# Feature Engineering

# DBSCAN

In [282]:
# Model Creation

In [283]:
# Hyperparameter Selection

In [284]:
# Model Training

# Multilayer Perceptron

In [285]:
# Model Creation

In [286]:
# Hyperparameter Selection

In [287]:
# Model Training

# Convolutiontional Neural Network

In [288]:
# Model Creation

In [289]:
# Hyperparameter Selection


In [290]:
# Model Training

# Model Visualization and Comparison

# Export model